# METODE ZA PODEŠAVANJE HIPERPARAMETARA

#Grid Search

In [1]:
!pip install scikeras
!pip install scikit-learn==1.4.2
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 86.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.4.2 which is incompatible.
hdbscan 0.8.41 requires scikit-learn>=1.6, but you have scikit-learn 1.4.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 8.2 MB/s eta 0:00:00


In [10]:
from tensorflow import keras
from tensorflow.keras import layers
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform
from skopt import BayesSearchCV
from sklearn.model_selection import PredefinedSplit
import pandas as pd
import plotly.graph_objects as go
from skopt.space import Real, Categorical, Integer

In [3]:
# Učitavanje Fashion-MNIST dataseta
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

print('Broj uzoraka u skupu za treniranje i testiranje:')
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print('\n')

class_names = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot"
]

print('Klase u datasetu:')
for i in range (len(class_names)):
    print(f"{i}: {class_names[i]}")

# Normalizacija (0–255 → 0–1)
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

# Pretvaranje slika 28x28 u vektore duljine 784
X_train = X_train.reshape(-1, 28 * 28)
X_test = X_test.reshape(-1, 28 * 28)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Broj uzoraka u skupu za treniranje i testiranje:
(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


Klase u datasetu:
0: T-shirt/top
1: Trouser
2: Pullover
3: Dress
4: Coat
5: Sandal
6: Shirt
7: Sneaker
8: Bag
9: Ankle boot


In [4]:
def create_model(neurons=128, learning_rate=0.001):
    model = keras.Sequential()

    model.add(layers.Dense(neurons, activation="relu", input_shape=(784,)))
    model.add(layers.Dense(neurons // 2, activation="relu"))
    model.add(layers.Dense(10, activation="softmax"))

    opt = keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(
        optimizer=opt,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model

#MANUAL SEARCH

In [76]:
model = KerasClassifier(model=create_model, epochs=10, verbose=0)

manual_configs = [
    {"neurons": 32,  "learning_rate": 0.01,   "batch_size": 32},
    {"neurons": 64,  "learning_rate": 0.001,  "batch_size": 64},
    {"neurons": 128, "learning_rate": 0.0001,  "batch_size": 32}
]

manual_results = []

for config in manual_configs:
    print(f"Testiram model: ", config)

    model.set_params(
        model__neurons=config['neurons'],
        model__learning_rate=config['learning_rate']
    )

    # Treniranje
    history = model.fit(
        X_train, y_train,
        epochs=10,
        batch_size=config['batch_size'],
        verbose=0,
        validation_split=0.2
    )

    # Evaluacija na testnom skupu
    score = model.score(X_test, y_test)
    manual_results.append(score)
    print(f"   Točnost: {score:.4f}")

best_manual_idx = np.argmax(manual_results)
best_manual_score = manual_results[best_manual_idx]

Testiram model:  {'neurons': 32, 'learning_rate': 0.01, 'batch_size': 32}


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



   Točnost: 0.8257
Testiram model:  {'neurons': 64, 'learning_rate': 0.001, 'batch_size': 64}


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



   Točnost: 0.8695
Testiram model:  {'neurons': 128, 'learning_rate': 0.0001, 'batch_size': 32}


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



   Točnost: 0.8647


PRIKAZ MS

In [77]:
neurons_m = np.array([c['neurons'] for c in manual_configs])
lr_m = np.array([c['learning_rate'] for c in manual_configs])
bs_m = np.array([c['batch_size'] for c in manual_configs])
manual_res_arr = np.array(manual_results)
labels_m = [f"N:{c['neurons']}, LR:{c['learning_rate']}, B:{c['batch_size']}" for c in manual_configs]

fig_3d_m = go.Figure()

# Batch size = 32
mask_32_m = bs_m == 32
if any(mask_32_m):
    fig_3d_m.add_trace(go.Scatter3d(
        x=neurons_m[mask_32_m],
        y=lr_m[mask_32_m],
        z=manual_res_arr[mask_32_m],
        mode='markers',
        marker=dict(size=8, color='blue'),
        name='Batch size = 32'
    ))

# Batch size = 64
mask_64_m = bs_m == 64
if any(mask_64_m):
    fig_3d_m.add_trace(go.Scatter3d(
        x=neurons_m[mask_64_m],
        y=lr_m[mask_64_m],
        z=manual_res_arr[mask_64_m],
        mode='markers',
        marker=dict(size=8, color='green'),
        name='Batch size = 64'
    ))


# Najbolja kombinacija
best_idx_m = np.argmax(manual_res_arr)
fig_3d_m.add_trace(go.Scatter3d(
    x=[neurons_m[best_idx_m]],
    y=[lr_m[best_idx_m]],
    z=[manual_res_arr[best_idx_m]],
    mode='text',
    text=['Optimum'],
    textposition='top center',
    name='Optimum'
))

fig_3d_m.update_layout(
    title='Manual Search',
    scene=dict(
        xaxis_title='Neurons',
        yaxis_title='Learning rate',
        zaxis_title='Accuracy'
    )
)
fig_3d_m.show()

In [78]:
print("Najbolja točnost - MS:")
best_idx_m = np.argmax(manual_results)
best_score_ms = manual_results[best_idx_m]
print(best_score_ms)

print("\nNajbolji hiperparametri - MS:")
best_params_ms = manual_configs[best_idx_m]
print(best_params_ms)

Najbolja točnost - MS:
0.8695

Najbolji hiperparametri - MS:
{'neurons': 64, 'learning_rate': 0.001, 'batch_size': 64}


#GRID SEARCH

In [44]:
model = KerasClassifier(model=create_model, epochs=10, verbose=0)

param_grid = {
    "model__neurons": [32, 64, 128],
    "model__learning_rate": [0.0001, 0.001, 0.01],
    "batch_size": [32, 64]
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring="accuracy", n_jobs=1)

grid_result = grid.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



PRIKAZ GS

In [45]:
# 3D prikaz rezultata grid searcha
params = grid_result.cv_results_['params']
scores = grid_result.cv_results_['mean_test_score']

neurons = np.array([p['model__neurons'] for p in params])
learning_rate = np.array([p['model__learning_rate'] for p in params])
accuracy = np.array(scores)
batch_size = np.array([p['batch_size'] for p in params])

fig = go.Figure()

# --- Batch size = 32 ---
mask_32 = batch_size == 32
fig.add_trace(go.Scatter3d(
    x=neurons[mask_32],
    y=learning_rate[mask_32],
    z=accuracy[mask_32],
    mode='markers',
    marker=dict(size=6, color='blue'),
    name='Batch size = 32'
))

# --- Batch size = 64 ---
mask_64 = batch_size == 64
fig.add_trace(go.Scatter3d(
    x=neurons[mask_64],
    y=learning_rate[mask_64],
    z=accuracy[mask_64],
    mode='markers',
    marker=dict(size=6, color='green'),
    name='Batch size = 64'
))

# --- Najbolja kombinacija ---
best_idx = np.argmax(accuracy)
fig.add_trace(go.Scatter3d(
    x=[neurons[best_idx]],
    y=[learning_rate[best_idx]],
    z=[accuracy[best_idx]],
    mode='text',
    text=['Optimum'],
    textposition='top center',
    name='Optimum'
))

fig.update_layout(
    title='Grid Search',
    scene=dict(
        xaxis_title='Neurons',
        yaxis_title='Learning rate',
        zaxis_title='Accuracy'
    )
)

fig.show()

In [46]:
print("Najbolja točnost - GS:")
print(grid_result.best_score_)

print("\nNajbolji hiperparametri - GS:")
print(grid_result.best_params_)

best_model_gs = grid_result.best_estimator_
loss_and_metrics_gs = best_model_gs.model_.evaluate(X_test, y_test)
test_loss_gs = loss_and_metrics_gs[0]
test_accuracy_gs = loss_and_metrics_gs[1]

print("\nTočnost na test skupu - GS:")
print(test_accuracy_gs)
print("Loss na test skupu - GS:")
print(test_loss_gs)

Najbolja točnost - GS:
0.88435

Najbolji hiperparametri - GS:
{'batch_size': 64, 'model__learning_rate': 0.001, 'model__neurons': 128}
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8759 - loss: 0.3578

Točnost na test skupu - GS:
0.8734999895095825
Loss na test skupu - GS:
0.3585924506187439


#RANDOMIZED SEARCH

In [73]:
model = KerasClassifier(model=create_model, epochs=10, verbose=0)

param_dist = {
    "model__neurons": [32, 64, 128],
    "model__learning_rate": loguniform(1e-4, 1e-2),
    "batch_size": [32, 64]
}

random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=20,
    cv=3,
    scoring="accuracy",
    n_jobs=1,
    random_state=42
)

random_result = random_search.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` objec

PRIKAZ RS

In [74]:
# Parametri i rezultati
params_rs = random_result.cv_results_['params']
scores_rs = random_result.cv_results_['mean_test_score']

neurons_rs = np.array([p['model__neurons'] for p in params_rs])
learning_rate_rs = np.array([p['model__learning_rate'] for p in params_rs])
accuracy_rs = np.array(scores_rs)
batch_size_rs = np.array([p['batch_size'] for p in params_rs])

fig_rs = go.Figure()

# Batch size = 32
mask_32_rs = batch_size_rs == 32
fig_rs.add_trace(go.Scatter3d(
    x=neurons_rs[mask_32_rs],
    y=learning_rate_rs[mask_32_rs],
    z=accuracy_rs[mask_32_rs],
    mode='markers',
    marker=dict(size=6, color='blue'),
    name='Batch size = 32'
))

# Batch size = 64
mask_64_rs = batch_size_rs == 64
fig_rs.add_trace(go.Scatter3d(
    x=neurons_rs[mask_64_rs],
    y=learning_rate_rs[mask_64_rs],
    z=accuracy_rs[mask_64_rs],
    mode='markers',
    marker=dict(size=6, color='green'),
    name='Batch size = 64'
))

# Najbolja kombinacija
best_idx_rs = np.argmax(accuracy_rs)
fig_rs.add_trace(go.Scatter3d(
    x=[neurons_rs[best_idx_rs]],
    y=[learning_rate_rs[best_idx_rs]],
    z=[accuracy_rs[best_idx_rs]],
    mode='text',
    text=['Optimum'],
    textposition='top center',
    name='Optimum'
))

fig_rs.update_layout(
    title='Random Search',
    scene=dict(
        xaxis_title='Neurons',
        yaxis_title='Learning rate',
        zaxis_title='Accuracy'
    )
)

fig_rs.show()

In [75]:
print("Najbolja točnost - RS:")
print(random_result.best_score_)

print("\nNajbolji hiperparametri - RS:")
print(random_result.best_params_)

best_model_rs = random_result.best_estimator_
loss_and_metrics_rs = best_model_rs.model_.evaluate(X_test, y_test)
test_loss_rs = loss_and_metrics_rs[0]
test_accuracy_rs = loss_and_metrics_rs[1]

print("\nTočnost na test skupu - RS:")
print(test_accuracy_rs)
print("Loss na test skupu - RS:")
print(test_loss_rs)

Najbolja točnost - RS:
0.88685

Najbolji hiperparametri - RS:
{'batch_size': 32, 'model__learning_rate': np.float64(0.0007792297153883001), 'model__neurons': 128}
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8866 - loss: 0.3373

Točnost na test skupu - RS:
0.8847000002861023
Loss na test skupu - RS:
0.33942243456840515


#BAYESIAN OPTIMIZATION

In [70]:
search_spaces = {
    'model__neurons': Integer(32, 128),
    'model__learning_rate': Real(1e-4, 1e-2, prior='log-uniform'),
    'batch_size': Categorical([32, 64])
}

bayes_search = BayesSearchCV(
    estimator=model,
    search_spaces=search_spaces,
    n_iter=20,
    cv=3,
    n_jobs=1,
    random_state=42
)

bayes_result = bayes_search.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` objec

#PRIKAZ BO

In [71]:
params_bo = bayes_result.cv_results_['params']
scores_bo = bayes_result.cv_results_['mean_test_score']

neurons_bo = np.array([p['model__neurons'] for p in params_bo])
learning_rate_bo = np.array([p['model__learning_rate'] for p in params_bo])
accuracy_bo = np.array(scores_bo)
batch_size_bo = np.array([p['batch_size'] for p in params_bo])

fig_bo = go.Figure()

# Batch size = 32
mask_32_bo = batch_size_bo == 32
fig_bo.add_trace(go.Scatter3d(
    x=neurons_bo[mask_32_bo],
    y=learning_rate_bo[mask_32_bo],
    z=accuracy_bo[mask_32_bo],
    mode='markers',
    marker=dict(size=6, color='blue'),
    name='Batch size = 32'
))

# Batch size = 64
mask_64_bo = batch_size_bo == 64
fig_bo.add_trace(go.Scatter3d(
    x=neurons_bo[mask_64_bo],
    y=learning_rate_bo[mask_64_bo],
    z=accuracy_bo[mask_64_bo],
    mode='markers',
    marker=dict(size=6, color='green'),
    name='Batch size = 64'
))

# Najbolje kombinacije
best_idx_bo = np.argmax(accuracy_bo)
fig_bo.add_trace(go.Scatter3d(
    x=[neurons_bo[best_idx_bo]],
    y=[learning_rate_bo[best_idx_bo]],
    z=[accuracy_bo[best_idx_bo]],
    mode='text',
    text=['Optimum'],
    textposition='top center',
    name='Optimum'
))

fig_bo.update_layout(
    title='Bayesian Optimization Results',
    scene=dict(
        xaxis_title='Neurons',
        yaxis_title='Learning rate',
        zaxis_title='Accuracy'
    )
)

fig_bo.show()

In [72]:
print("Najbolja točnost - BO:")
print(bayes_result.best_score_)

print("\nNajbolji hiperparametri - BO:")
print(bayes_result.best_params_)

best_model_bo = bayes_result.best_estimator_
loss_and_metrics = best_model_bo.model_.evaluate(X_test, y_test, verbose=0)
test_loss_bo = loss_and_metrics[0]
test_accuracy_bo = loss_and_metrics[1]

print("\nTočnost na test skupu - BO:")
print(test_accuracy_bo)
print("Loss na test skupu - BO:")
print(test_loss_bo)

Najbolja točnost - BO:
0.8822333333333333

Najbolji hiperparametri - BO:
OrderedDict({'batch_size': 32, 'model__learning_rate': 0.0005532606478682793, 'model__neurons': 128})

Točnost na test skupu - BO:
0.8855999708175659
Loss na test skupu - BO:
0.32589322328567505
